In [1]:
import os

In [2]:
import numpy as np
import pandas as pd

df = pd.read_excel('data.xls')

empList = df.to_dict(orient='records')
empList.pop()

uniqer = set()
for emp in empList:
    uniqer.add(emp['accountNo'])

# for emp in empList:
#     if emp['employeeDeploySite'] == 'E': print(emp['employeeName'])

for emp in empList:
    del emp['BRANCH']
    
    emp['empId'] = str(int(emp['empId']))
    empType = emp['employeeTypeId']
    if empType == "HIGHLY SKILLED": empType = 1
    elif empType == "SKILLED": empType = 2
    else: empType = 3

    emp['employeeTypeId'] = empType
    emp['contact'] = None
    emp['emergencyContact'] = None
    emp['email'] = None
    emp['pfNo'] = None

    try:
        emp['pfuanNo'] == int(emp['pfuanNo'])
    except:
        emp['pfuanNo'] = None
    
    emp['esicNo'] = 1223344556

    try:
        emp['adharNo'] == str(int(emp['adharNo']))
    except:
        emp['adharNo'] = None
    
    emp['panNo'] = None

    emp['safetyPassNo'] = None if type(emp['safetyPassNo']) != str else emp['safetyPassNo']

    emp['bankName'] = None

    try:
        emp['accountNo'] = int(emp['accountNo'])
    except:
        emp['accountNo'] = None

    try:
        emp['adharNo'] == str(int(emp['adharNo']))
    except:
        emp['adharNo'] = None

    try:
        emp['incentive'] = int(emp['incentive'])
    except:
        emp['incentive'] = 0

    emp['ifscCode'] = None if type(emp['ifscCode']) != str else emp['ifscCode']

    emp['jntvtiCertNo'] = None
    emp['jntvtiCategory'] = None
    emp['workExp'] = 0

    try:
        a = int(emp['actualRate'])
    except:
        emp['actualRate'] = 0.0
    
    try:
        emp['fixed_rate'] = int(emp['fixed_rate'])
    except:
        emp['fixed_rate'] = 0
    
    emp['otType'] = 0

    if type(emp['workingHours']) == str:
        emp['workingHours'] = int(emp['workingHours'].split('hrs')[0])
    else:
        emp['workingHours'] = 8
    
    emp['qualification'] = None

    emp['dob'] = None

    emp['payment_type'] = 'Fixed' if emp['fixed_rate'] != 0 else 'Actual'

    emp['gender'] = None

# for uniq in uniqer:
#     print(uniq)

In [94]:
# uniqer = set()
# for emp in empList:
#     uniqer.add(emp['accountNo'])

# for uniq in uniqer:
#     print(uniq)

In [3]:
# pd.DataFrame(list(empList[0].keys()), list(empList[0].values()))
list(empList[0].keys())
columns = "("
values = "("

for key in empList[0].keys():
    columns += key + ", "
    values += "%s, "

columns = columns[0:-2]
values = values[0:-2]

columns += ")"
values += ")"

In [4]:
import pymysql

HOST = os.environ.get("DB_HOST")
USER = os.environ.get("DB_USER")
PASSWORD = os.environ.get("DB_PASSWORD")
DATABASE = os.environ.get("DB_SCHEMA")

connection = pymysql.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE,
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        # sql = 'SELECT * FROM information_schema.tables WHERE table_type=\'BASE TABLE\''
        # sql = 'DESCRIBE attendance_record'
        sql = f'INSERT INTO employees {columns} VALUE {values}'

        inserted = []
        for emp in empList:
            cursor.execute(f'SELECT * FROM employees WHERE empId=\'{emp['empId']}\'')
            exist = cursor.fetchone()
            if exist: continue

            cursor.execute(sql, list(emp.values()))
            inserted.append(emp)
        
        connection.commit()
        
        print("inserted", len(inserted), "employees")

inserted 109 employees


In [5]:
import csv

with open('test.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, list(inserted[0].keys()))
    writer.writeheader()
    writer.writerows(inserted)

In [109]:
len(empList)

148